add train noise

In [ ]:
! gdown https://drive.google.com/uc?id=1AbxjCBByF-xGV3c7FJ1ztZC1HO0O0uJ0

Downloading...
From: https://drive.google.com/uc?id=1AbxjCBByF-xGV3c7FJ1ztZC1HO0O0uJ0
To: /content/kaggle.json
100% 66.0/66.0 [00:00<00:00, 249kB/s]


In [ ]:
! pip install kaggle
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content/'
! kaggle competitions download -c comp-551-fall-2021
! unzip images_l.pkl.zip
! unzip images_test.pkl.zip
! unzip images_ul.pkl.zip
! unzip labels_l.pkl.zip

 87% 86.0M/99.0M [00:00<00:00, 192MB/s]
100% 99.0M/99.0M [00:00<00:00, 223MB/s]
  0% 0.00/141k [00:00<?, ?B/s]
100% 141k/141k [00:00<00:00, 126MB/s]
 91% 179M/197M [00:00<00:00, 186MB/s]
100% 197M/197M [00:01<00:00, 197MB/s]
 91% 181M/198M [00:00<00:00, 189MB/s]
100% 198M/198M [00:01<00:00, 195MB/s]
Archive:  images_l.pkl.zip
  inflating: images_l.pkl            
Archive:  images_test.pkl.zip
  inflating: images_test.pkl         
Archive:  images_ul.pkl.zip
  inflating: images_ul.pkl           
Archive:  labels_l.pkl.zip
  inflating: labels_l.pkl            


In [ ]:
import io
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, LeakyReLU, ZeroPadding2D,Convolution2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import History
from tensorflow.keras.optimizers import Nadam, Adam, SGD
from keras.applications.vgg16 import VGG16
from keras.applications.densenet import DenseNet169
from keras.applications.resnet import ResNet152, ResNet101, ResNet50
import matplotlib.pyplot as plt
from keras.utils import np_utils
from tensorflow.keras import backend as K
from tensorflow.keras import regularizers

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import OneHotEncoder

import tensorflow as tf
from google.colab import drive
import numpy as np
import h5py
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras import layers, regularizers, Input
from keras.layers import *
from keras.callbacks import ReduceLROnPlateau
from keras import optimizers
import random

In [ ]:
# Image data for training (30,000 sample, each sample is a 56x56 image)
# train_labels = pd.read_csv('train_max_y.csv')
# train_images = pd.read_pickle('train_max_x')
# test_images = pd.read_pickle('test_max_x')
with open("images_l.pkl", 'rb') as f:
    # load into as a numpy array
    train_labelled_data = pickle.load(f)
# Test images. The prediction corresponding to these images should be uploaded. (15,000 samples)
with open("images_test.pkl", 'rb') as f:
    test_data = pickle.load(f)
# Labels for training (30,000 rows, each row is a size 36 binary vector, which is the label to the corresponding image)
with open("labels_l.pkl", 'rb') as f:
    train_labelled_y = pickle.load(f)
# Additional images that can be used for training the classifier.
# Labels for these images are not provided. (30,000 samples, where each sample is a 56x56 image)
with open("images_ul.pkl", 'rb') as f:
    train_unlabelled_x = pickle.load(f)

# Pre processing

In [ ]:
# data pre processing
def clear_noise (array):
  for n in range (0,array.shape[0]):
    for i in range (0,56):
      for j in range (0,56):
        # pixel of the nth image at position (i,j)
        pixel = array[n][i][j]
        flag = False
        if(i == 0):
        # at the left edge  
          if(j == 0):
          # at the top edge, so only count right and below
            if(array[n][i+1][j]>0 or array[n][i][j+1]>0):
              flag=True
          # at the bottom edge, so only count right and above
          elif(j == 55):
            if(array[n][i+1][j]>0 or array[n][i][j-1]>0):
              flag=True
          else:
          # count right, above and below
            if(array[n][i+1][j]>0 or array[n][i][j+1]>0 or array[n][i][j-1]>0):
              flag=True
        elif(i == 55):
        # at the right edge  
          if(j == 0):
            if(array[n][i-1][j]>0 or array[n][i][j+1]>0):
              flag=True
          elif(j == 55):
            if(array[n][i-1][j]>0 or array[n][i][j-1]>0):
              flag=True
          else:
            if(array[n][i-1][j]>0 or array[n][i][j+1]>0 or array[n][i][j-1]>0):
              flag=True
        else:
          if(j == 0):
            if(array[n][i+1][j]>0 or array[n][i-1][j]>0 or array[n][i][j+1]>0):
              flag=True
          elif(j == 55):
            if(array[n][i+1][j]>0 or array[n][i-1][j]>0 or array[n][i][j-1]>0):
              flag=True
          else:
            if(array[n][i+1][j]>0 or array[n][i-1][j]>0 or array[n][i][j+1]>0 or array[n][i][j-1]>0):
              flag=True   
        if(flag==False):
          # the point is supposed to not link with other point, so this pixel is regarded as noise
          array[n][i][j]=0
        # strengthen the useful pixel, make the background to be dark
        if(array[n][i][j]>70):
          array[n][i][j]=255
        else:
          array[n][i][j]=0
  return array

In [ ]:
def add_noise(array):
  for n in range (0,array.shape[0]):
    count = 0
    iteration = 0
    while (count<100 and iteration<56*56):
      i = random.randint(0,55)
      j = random.randint(0,55)
      pixel = array[n][i][j]
      if(pixel<50):
        array[n][i][j] = 175
        count = count+1
      iteration = iteration+1
  return array

In [ ]:
# train_labelled_x = train_labelled_data
# plt.imshow(train_labelled_x[2553], cmap="gray")
# plt.show()
# plt.imshow(train_labelled_x[142], cmap="gray")
# plt.show()

# train_labelled_x = add_noise(train_labelled_x)
# plt.imshow(train_labelled_x[2553], cmap="gray")
# plt.show()
# plt.imshow(train_labelled_x[142], cmap="gray")
# plt.show()

In [ ]:
# train_labelled_data = clear_noise(train_labelled_data)
train_labelled_data = add_noise(train_labelled_data)
test_data = clear_noise(test_data)

In [ ]:
# train_labelled_data=train_labelled_data.reshape((30000,-1))
# print(train_labelled_data.shape)
# mydf=pd.DataFrame(train_labelled_data)
# mydf.to_csv('processed_train_set.csv',index=False)

train_labelled_data=train_labelled_data.reshape((30000,-1))
print(train_labelled_data.shape)
mydf=pd.DataFrame(train_labelled_data)
mydf.to_csv('noise_train_set.csv',index=False)

test_data=test_data.reshape((15000,-1))
print(test_data.shape)
mydf=pd.DataFrame(test_data)
mydf.to_csv('processed_test_set.csv',index=False)

(30000, 3136)
(15000, 3136)


# Digit Prediction

In [ ]:
def cnn():
    # create model
    model = Sequential()
    vgg16 =  VGG16(weights='imagenet', include_top=False, input_shape=(56,56,3))
    #res = ResNet152(weights='imagenet', include_top=False, input_shape=(56,56,3))
# des = DenseNet169(weights='imagenet', include_top=False, input_shape=(56,56,3))
    model.add(vgg16)
# model.add(des)
# model.add(res)
    model.add(Flatten())
    model.add(Dense(1000, activation='relu'))
    # model.add(Dropout(0.5))
    model.add(Dropout(0.25))
    model.add(Dense(10, activation='softmax'))

    model.compile(loss='categorical_crossentropy',optimizer=keras.optimizers.Adam(lr=0.0001),metrics=['accuracy'])
    # Compile
    #model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = cnn()
model.summary() 

58900480/58889256 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 1000)              513000    
                                                                 
 dropout (Dropout)           (None, 1000)              0         
                                                                 
 dense_1 (Dense)             (None, 10)                10010     
                                                                 
Total params: 15,237,698
Trainable params: 15,237,698
Non-trainable params: 0
_____________________________________________

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
# train_labelled_x=np.array(pd.DataFrame(pd.read_csv('processed_train_set.csv')))

# train_labelled_x=train_labelled_x.reshape((30000,56,56))

# import matplotlib.pyplot as plt
# import random

# plt.imshow(train_labelled_x[random.randint(0,29999)], cmap="gray")
# plt.show()

# # no processed train set
# with open("images_l.pkl", 'rb') as f:
#     # load into as a numpy array
#     train_labelled_data = pickle.load(f)
# train_labelled_x1 = train_labelled_data

# noise train set 
train_labelled_x=np.array(pd.DataFrame(pd.read_csv('noise_train_set.csv')))
train_labelled_x=train_labelled_x.reshape((30000,56,56))


# train_labelled_x = np.concatenate((train_labelled_x1, train_labelled_x2), axis=0)
# print(train_labelled_x.shape)
# train_labelled_y = np.concatenate((train_labelled_y, train_labelled_y), axis=0)
# print(train_labelled_y.shape)

In [ ]:
x_train = train_labelled_x
y_train = train_labelled_y
for i in range(x_train.shape[0]):
  x_train[i] = (x_train[i]>200).astype('int32')*255
import torch

x_train = torch.Tensor(x_train)
x_train = torch.unsqueeze(x_train, dim=3)/255.
x_train = x_train.repeat(1,1,1,3)
x_train = x_train.numpy()
x_train = x_train.astype('float32')
x_valid = x_train[28000:, :]
x_train = x_train[:28000, :]
print('x_train shape:', x_train.shape)
print('x_valid shape:', x_valid.shape)

# y_train = keras.utils.to_categorical(y_train, 10)
a = y_train
y_valid = y_train[28000:, :10]
y_train = y_train[:28000, :10]
print('y_train shape:', y_train.shape)
print('y_valid shape:', y_valid.shape)

x_train shape: (28000, 56, 56, 3)
x_valid shape: (2000, 56, 56, 3)
y_train shape: (28000, 10)
y_valid shape: (2000, 10)


In [ ]:
for i in range(15):
  print('Trail: ',i)
  history = model.fit(x_train, y_train, batch_size=100, epochs=1, verbose=1)
  score = model.evaluate(x_valid, y_valid, verbose=1)
  print('Test loss:', score[0])
  print('Test accuracy:', score[1])
  model.save('VGG16_2_SP_{}'.format(i))

Trail:  0
63/63 [==============================] - 1s 14ms/step - loss: 0.6059 - accuracy: 0.8200
Test loss: 0.6059260368347168
Test accuracy: 0.8199999928474426
INFO:tensorflow:Assets written to: VGG16_2_SP_0/assets
Trail:  1
63/63 [==============================] - 1s 11ms/step - loss: 0.4704 - accuracy: 0.8595
Test loss: 0.470396488904953
Test accuracy: 0.859499990940094
INFO:tensorflow:Assets written to: VGG16_2_SP_1/assets
Trail:  2
63/63 [==============================] - 1s 11ms/step - loss: 0.4326 - accuracy: 0.8670
Test loss: 0.4325747787952423
Test accuracy: 0.8669999837875366
INFO:tensorflow:Assets written to: VGG16_2_SP_2/assets
Trail:  3
63/63 [==============================] - 1s 11ms/step - loss: 0.4056 - accuracy: 0.8715
Test loss: 0.40556076169013977
Test accuracy: 0.8715000152587891
INFO:tensorflow:Assets written to: VGG16_2_SP_3/assets
Trail:  4
63/63 [==============================] - 1s 11ms/step - loss: 0.4263 - accuracy: 0.8775
Test loss: 0.4262731373310089
Test 

In [ ]:
history = model.fit(x_train, y_train, batch_size=100, epochs=1, verbose=1)
score = model.evaluate(x_valid, y_valid, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
# load testing data for model testing
x_test=np.array(pd.DataFrame(pd.read_csv('processed_test_set.csv')))

x_test=x_test.reshape((15000,56,56))

import matplotlib.pyplot as plt
import random

plt.imshow(x_test[random.randint(0,14999)], cmap="gray")
plt.show()

# with open("images_test.pkl",'rb') as f:
#   x_test = pickle.load(f)
for i in range(x_test.shape[0]):
  x_test[i] = (x_test[i]>200).astype('int32')*255
x_test = torch.Tensor(x_test)
x_test = torch.unsqueeze(x_test,dim=3)/255
x_test = x_test.repeat(1,1,1,3)
x_test = x_test.numpy()
x_test = x_test.astype('float32')

# letter

In [ ]:
def cnn2():
  model = Sequential()
  model.add(Input(shape=(56, 56, 1)))
  model.add(Conv2D(filters= 56, kernel_size=(3,3), padding='Same', activation='relu'))
  model.add(Conv2D(filters= 56, kernel_size=(3,3), padding='Same', activation='relu'))
  model.add(BatchNormalization())
  model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
  model.add(Dropout(0.6))

  model.add(Conv2D(filters= 128, kernel_size=(3,3), padding='Same', activation='relu'))
  model.add(Conv2D(filters= 128, kernel_size=(3,3), padding='Same', activation='relu'))
  model.add(BatchNormalization())
  model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
  model.add(Dropout(0.6))

  model.add(Conv2D(filters= 256, kernel_size=(3,3), padding='Same', activation='relu'))
  model.add(Conv2D(filters= 256, kernel_size=(3,3), padding='Same', activation='relu'))
  model.add(BatchNormalization())
  model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
  model.add(Dropout(0.6))

  model.add(Conv2D(filters= 512, kernel_size=(3,3), padding='Same', activation='relu'))
  model.add(Conv2D(filters= 512, kernel_size=(3,3), padding='Same', activation='relu'))
  model.add(BatchNormalization())
  model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
  model.add(Dropout(0.6))

  model.add(Flatten())
  model.add(Dense(1024, activation='relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.6))
  model.add(Dense(26, activation='softmax'))
  model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])
  return model
model2=cnn2()
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 56, 56, 56)        560       
                                                                 
 conv2d_1 (Conv2D)           (None, 56, 56, 56)        28280     
                                                                 
 batch_normalization (BatchN  (None, 56, 56, 56)       224       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 28, 28, 56)       0         
 )                                                               
                                                                 
 dropout_1 (Dropout)         (None, 28, 28, 56)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 28, 128)      

In [ ]:
learnrate_reduce_1 = ReduceLROnPlateau(monitor='val_dense_1_accuracy', patience=2, verbose=1,factor=0.8, min_lr=0.000001)
learnrate_reduce_2 = ReduceLROnPlateau(monitor='val_dense_3_accuracy', patience=2, verbose=1,factor=0.8, min_lr=0.000001)
learnrate_reduce_3 = ReduceLROnPlateau(monitor='val_dense_5_accuracy', patience=2, verbose=1,factor=0.8, min_lr=0.000001)
learnrate_reduce_4 = ReduceLROnPlateau(monitor='val_dense_7_accuracy', patience=2, verbose=1,factor=0.8, min_lr=0.000001)
learnrate_reduce_5 = ReduceLROnPlateau(monitor='val_dense_9_accuracy', patience=2, verbose=1,factor=0.8, min_lr=0.000001)

In [ ]:
# reload train data for model2 training
train_labelled_x=np.array(pd.DataFrame(pd.read_csv('noise_train_set.csv')))

train_labelled_x=train_labelled_x.reshape((30000,56,56))

# with open("images_l.pkl", 'rb') as f:
#     # load into as a numpy array
#     train_labelled_data = pickle.load(f)

# Labels for training (30,000 rows, each row is a size 36 binary vector, which is the label to the corresponding image)
with open("labels_l.pkl", 'rb') as f:
    train_labelled_y = pickle.load(f)

train_labelled_x = train_labelled_data

In [ ]:
letter_train = train_labelled_x.reshape(-1, 56, 56, 1)
# letter_train /= 255
letter_train_label = train_labelled_y[:, 10:]
x_train2 = letter_train[:29000,:,:,:]
y_train2 = letter_train_label[:29000,:]
x_valid2 = letter_train[29000:,:,:,:]
y_valid2 = letter_train_label[29000:,:]

In [ ]:
history = model2.fit(x_train2, y_train2, epochs=40, batch_size=128,
          validation_data=(x_valid2, y_valid2), 
          callbacks=[learnrate_reduce_1], shuffle=False)

Epoch 1/40
227/227 [==============================] - 13s 56ms/step - loss: 0.1420 - accuracy: 0.9559 - val_loss: 0.1662 - val_accuracy: 0.9620 - lr: 0.0010
Epoch 2/40
227/227 [==============================] - 13s 56ms/step - loss: 0.1356 - accuracy: 0.9573 - val_loss: 0.1552 - val_accuracy: 0.9670 - lr: 0.0010
Epoch 3/40
227/227 [==============================] - 13s 56ms/step - loss: 0.1308 - accuracy: 0.9599 - val_loss: 0.3205 - val_accuracy: 0.9220 - lr: 0.0010
Epoch 4/40
227/227 [==============================] - 13s 56ms/step - loss: 0.1234 - accuracy: 0.9607 - val_loss: 0.1550 - val_accuracy: 0.9670 - lr: 0.0010
Epoch 5/40
227/227 [==============================] - 13s 56ms/step - loss: 0.1203 - accuracy: 0.9622 - val_loss: 0.1306 - val_accuracy: 0.9580 - lr: 0.0010
Epoch 6/40
227/227 [==============================] - 13s 56ms/step - loss: 0.1261 - accuracy: 0.9608 - val_loss: 0.2044 - val_accuracy: 0.9470 - lr: 0.0010
Epoch 7/40
227/227 [==============================] - 13s 

In [ ]:
# # load testing data for model testing
# x_test_data=np.array(pd.DataFrame(pd.read_csv('processed_test_set.csv')))

# x_test_data=x_test_data.reshape((15000,56,56))

# import matplotlib.pyplot as plt
# import random

# plt.imshow(x_test_data[random.randint(0,14999)], cmap="gray")
# plt.show()

with open("images_test.pkl",'rb') as f:
  x_test_data = pickle.load(f)
x_test = x_test_data.reshape(-1,56,56,1)
# x_test /= 255

In [ ]:
result2 = model2.predict(x_test, verbose=1)
result2 = np.argmax(result2, axis = 1)
print(result2[0:10])